In [2]:
# -----------------------------------------------------------------------------
# Copyright 2020 by ShabaniPy Authors, see AUTHORS for more details.
#
# Distributed under the terms of the MIT license.
#
# The full license is in the file LICENCE, distributed with this software.
# -----------------------------------------------------------------------------
"""Example of using data classification tools for Shapiro experiments.

"""
#: Path the raw data agglomerate
RAW_DATA = (
    "/Users/bh/Desktop/Research/DARPA/Data Aggregation and Processing/"
    "JS311/MG2/General/JS311-MG2_raw_data-Gen.hdf5"
)

#: Path into which to duplicate the raw data before starting preprocessing.
DUPLICATION_PATH = (
    "/Users/bh/Desktop/Research/DARPA/Data Aggregation and Processing/"
    "JS311/MG2/General/JS311-MG2_pre-processed_data-Gen.hdf5"
)

#: Path into which to save the processed data output.
PROCESSING_PATH = (
    "/Users/bh/Desktop/Research/DARPA/Data Aggregation and Processing/"
    "JS311/MG2/General/JS311-MG2_processed_data-Gen.hdf5"
)

#: Directory in which to save the summary plots.
SUMMARY_DIRECTORY = (
    "/Users/bh/Desktop/Research/DARPA/Data Aggregation and Processing/"
    "JS311/MG2/General/generated-figs"
)

#: Path to a file in which to store teh log of teh aggregation
LOG_PATH = (
    "/Users/bh/Desktop/Research/DARPA/Data Aggregation and Processing/Log/"
    "JS311-MG2/JS311-MG2_aggregation.log"
)

import logging
import os

import h5py
import matplotlib
from shabanipy.jj.iv_analysis import analyse_vi_curve, extract_critical_current
from shabanipy.jj.shapiro.binning import bin_power_shapiro_steps
from shabanipy.jj.shapiro.plotting import (
    plot_differential_resistance_map,
    plot_shapiro_histogram,
    plot_step_weights,
)
from shabanipy.jj.shapiro.utils import correct_voltage_offset_per_power
from shabanipy.jj.utils import compute_resistance, correct_voltage_offset
from shabanipy.utils.data_processing import (
    PreProcessingStep,
    ProcessCoordinator,
    ProcessingStep,
    SummarizingStep,
)

matplotlib.use("Agg")


# Parametrize the logging system
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
ch = logging.StreamHandler(None)
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)
ch = logging.FileHandler(LOG_PATH)
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)

# Avoid matplotlib debug messages
mpl_logger = logging.getLogger("matplotlib")
mpl_logger.setLevel(logging.WARNING)

# Get large bias measurements and Shapiro with a minimum number of points in power
coor = ProcessCoordinator(
    archive_path=RAW_DATA,
    duplicate_path=DUPLICATION_PATH,
    processing_path=PROCESSING_PATH,
    summary_directory=SUMMARY_DIRECTORY,
    preprocessing_steps=[
        PreProcessingStep(
            name="bias_scaling",
            measurements=["IcRn",
                          #"Shapiro"
                         ],
            input_quantities=["Bias"],
            parameters={"scaling": 1},
            routine=lambda bias, scaling, debug: bias * scaling,
            output_quantities=["ScaledBias"],
        ),
#         PreProcessingStep(
#             name="jj_curr_scaling",
#             measurements=["IcRn","Shapiro"],
#             input_quantities=["JJ Current"],
#             parameters={"scaling": 1/50},
#             routine=lambda jjcurr, scaling, debug: jjcurr * scaling,
#             output_quantities=["ScaledJJCurrent"],
#         ),
        PreProcessingStep(
            name="voltage_scaling",
            measurements=["IcRn",
                          #"Shapiro"
                         ],
            input_quantities=["Voltage Drop"],
            parameters={"scaling": 1},
            routine=lambda volt, scaling, debug: volt * scaling,
            output_quantities=["ScaledVoltage"],
        ),
#         PreProcessingStep(
#             name="voltage_offset_coarse",
#             measurements=["IcRn"],
#             input_quantities=["ScaledBias", "ScaledVoltage"],
#             #parameters={},
#             #routine=lambda bias, volt, debug: volt - 0.0023,
#             parameters={"n_peak_width": 2, "bound":6},
#             routine=correct_voltage_offset,
#             output_quantities=["VoltageCoarseOffset"],
#         ),
#         PreProcessingStep(
#             name="voltage_offset_coarse",
#             measurements=["Shapiro"],
#             input_quantities=["ScaledBias", "ScaledVoltage"],
#             parameters={"n_peak_width": 2, "index": 0,"bound":7.5},
#             routine=correct_voltage_offset,
#             output_quantities=["VoltageCoarseOffset"],
#         ),
#         PreProcessingStep(
#             name="voltage_offset_fine",
#             measurements=["Shapiro"],
#             input_quantities=["RF Power", "ScaledBias", "VoltageCoarseOffset"],
#             parameters={
#                 "frequency": "classifiers@1@Frequency",
#                 "n_peak_width": 2,
#                 "n_std_as_bin": 1,
#                 "bound": 7.5
#             },
#             routine=correct_voltage_offset_per_power,
#             output_quantities=["VoltageFineOffset"],
#         ),
    ],
    processing_steps=[
#         ProcessingStep(
#             name="ic_rn",
#             input_origin="raw@IcRn",
#             input_quantities=["ScaledBias", "ScaledVoltage"],
#             parameters={"ic_voltage_threshold": 1e-7, "high_bias_threshold":0.5e-6},
#             routine=analyse_vi_curve,
#             tier="0",
#             output_quantities=["Rn_c", "Rn_h", "Ic_c", "Ic_h", "Ie_c", "Ie_h"],
#         ),
        ProcessingStep(
            name="ic_rn",
            input_origin="raw@IcRn",
            input_quantities=["ScaledBias", "ScaledVoltage"],
            parameters={},
            routine=extract_critical_current,
            tier="0",
            output_quantities=["ic_c"],
        ),
        # Copy the RF power over the processed quantity since it is needed for
        # summary
#         ProcessingStep(
#             name="copy_rf_power",
#             input_origin="raw@Shapiro",
#             input_quantities=["RF Power"],
#             parameters={},
#             routine=lambda rf, debug: rf,
#             tier="0",
#             output_quantities=["RF Power"],
#         ),
#         ProcessingStep(
#             name="derivative",
#             input_origin="raw@Shapiro",
#             input_quantities=["ScaledBias", "VoltageFineOffset"],
#             parameters={},
#             routine=compute_resistance,
#             tier="0",
#             output_quantities=["DifferentialResistance_Bias", "DifferentialResistance"],
#         ),
#         ProcessingStep(
#             name="shapiro_binning",
#             input_origin="raw@Shapiro",
#             input_quantities=["RF Power", "ScaledBias", "VoltageFineOffset"],
#             parameters={"frequency": "classifiers@1@Frequency", "step_fraction": 0.1},
#             routine=bin_power_shapiro_steps,
#             tier="0",
#             output_quantities=["VoltageBins", "HistogramCounts"],
#         ),
    ],
    summary_steps=[
#         SummarizingStep(
#             name="plot_resistance",
#             input_quantities=[
#                 "RF Power",
#                 "DifferentialResistance_Bias",
#                 "DifferentialResistance",
#             ],
#             parameters={},
#             routine=plot_differential_resistance_map,
#             tier="0",
#             use_named_subdirectory=True,
#         ),
#         SummarizingStep(
#             name="plot_histo",
#             input_quantities=["RF Power", "VoltageBins", "HistogramCounts",],
#             parameters={"mark_steps": [0, -1, -2], "mark_steps_limit": 0.1,"I_c":5.5,
#                         "voltage_limit":4.0, "transpose":True,},
#             routine=plot_shapiro_histogram,
#             tier="0",
#             use_named_subdirectory=True,
#         ),
#         SummarizingStep(
#             name="plot_weights",
#             input_quantities=["RF Power", "VoltageBins", "HistogramCounts",],
#             parameters={"steps": [0, -1, -2], "ic": 5.5},
#             routine=plot_step_weights,
#             tier="0",
#             use_named_subdirectory=True,
#         ),
    ],
)

coor.run_preprocess()

coor.run_process()

coor.run_summary()



Copying /Users/bh/Desktop/Research/DARPA/Data Aggregation and Processing/JS311/MG2/General/JS311-MG2_raw_data-Gen.hdf5 to /Users/bh/Desktop/Research/DARPA/Data Aggregation and Processing/JS311/MG2/General/JS311-MG2_pre-processed_data-Gen.hdf5
Copying /Users/bh/Desktop/Research/DARPA/Data Aggregation and Processing/JS311/MG2/General/JS311-MG2_raw_data-Gen.hdf5 to /Users/bh/Desktop/Research/DARPA/Data Aggregation and Processing/JS311/MG2/General/JS311-MG2_pre-processed_data-Gen.hdf5
Copied /Users/bh/Desktop/Research/DARPA/Data Aggregation and Processing/JS311/MG2/General/JS311-MG2_raw_data-Gen.hdf5 to /Users/bh/Desktop/Research/DARPA/Data Aggregation and Processing/JS311/MG2/General/JS311-MG2_pre-processed_data-Gen.hdf5
Copied /Users/bh/Desktop/Research/DARPA/Data Aggregation and Processing/JS311/MG2/General/JS311-MG2_raw_data-Gen.hdf5 to /Users/bh/Desktop/Research/DARPA/Data Aggregation and Processing/JS311/MG2/General/JS311-MG2_pre-processed_data-Gen.hdf5
Running pre-processing: bias_s

(0,) [ 7 22 48 60 85] [13 49]
(1,) [38 65 87] [11 45 93]
(2,) [53 65 88] [10 30 50]
(3,) [ 7 18 37 49 66 71 90] [ 8 81]
(4,) [ 2 29 47 70 92] [ 6 33]
(5,) [45 80 93] [ 5 29 92]
(6,) [60 93] [ 5 20 44 91]
(7,) [52 91] [ 6 24 56 93]
(8,) [39 89] [ 8 24 35 56 66 92]
(9,) [ 7 17 62 88] [10 33 49 73]
(10,) [16 29 65 86] [11 92]
(11,) [57 85] [13 42 50 92]
(12,) [14 27 65 84] [14 52 93]
(13,) [ 5 44 53 84] [15 53 76]
(14,) [ 8 84] [15 51]
(15,) [ 6 49 84] [15 53]
(16,) [ 7 84] [14 43]
(17,) [40 65 85] [13 43 64 82 94]
(18,) [35 60 87] [12 36]
(19,) [ 6 13 60 88] [10 35 60]
(20,) [ 7 21 47 62 90] [ 8 34]
(21,) [65 92] [ 6 24 40 91]
(22,) [41 76 94] [ 4 31 64 94]
(23,) [22 36 76 93] [ 5 26 81 93]
(24,) [ 6 55 91] [ 6 71]
(25,) [43 89] [ 9 41 55 92]
(26,) [60 87] [11 35 93]
(27,) [ 6 61 85] [13 35 55]
(28,) [83] [15 56 92]
(29,) [ 5 42 82] [16 55]
(30,) [10 42 53 81] [18 54 86]
(31,) [41 80] [18 53 93]
(32,) [ 7 49 80] [18]
(33,) [ 6 38 62 81] [18 56]
(34,) [ 3 82] [17 37 56 95]
(35,) [39 60 83

    for {0: {'sample': '2JJ-2HB-5MGJJ-MG2'}, 1: {'CM: GV': -0.5, 'RM+LM: GV': -3.0}}
    for {0: {'sample': '2JJ-2HB-5MGJJ-MG2'}, 1: {'CM: GV': -0.5, 'RM+LM: GV': -3.0}}


(22,) [21 44 55 65 76 93] [ 5 23 92]
(23,) [91] [ 7 92]
(24,) [ 7 64 89] [ 9 34 58 70]
(25,) [64 87] [11 34 53 93]
(26,) [ 4 59 64 85] [14 33 53]
(27,) [44 55 83] [16 34 56 82 92]
(28,) [ 2 44 82] [17 55 93]
(29,) [45 81] [18 35 56 83]
(30,) [ 5 18 43 80] [19 56 83]
(31,) [44 80] [19 54 95]
(32,) [ 6 43 80] [19 53]
(33,) [52 81] [18 44 54 94]
(34,) [ 2 18 64 82] [16 44]
(35,) [ 8 84] [14 37 64 86]
(36,) [ 8 41 86] [12 44 53 81]
(37,) [19 41 53 88] [10 32 90]
(38,) [ 6 50 66 91] [ 7 21 46 95]
(39,) [41 92] [ 5 90]
(40,) [75 92] [ 6 60 92]
(41,) [ 9 90] [8]
(42,) [69 87] [11 38 55 94]
(43,) [85] [14 37 58 92]
(44,) [42 58 82] [17 39 58 87 98]
(45,) [ 9 39 59 80] [19 38 59]
(46,) [ 8 41 58 78] [21 41 57]
(47,) [38 59 77] [23 58 89]
(48,) [ 7 40 76] [23 59 77]
(49,) [40 76] [24 40 61 92]
(50,) [38 76] [23 39 61 90]
(51,) [38 77] [22 40 49 59 92]
(52,) [39 78] [20 60 93]
(53,) [11 31 80] [18 42 62]
(54,) [ 6 39 58 83] [15 40 89]
(55,) [30 57 71 86] [12 67 92]
(56,) [52 70 89] [ 9 27 86]
(57

    for {0: {'sample': '2JJ-2HB-5MGJJ-MG2'}, 1: {'CM: GV': -0.5, 'RM+LM: GV': -4.0}}
    for {0: {'sample': '2JJ-2HB-5MGJJ-MG2'}, 1: {'CM: GV': -0.5, 'RM+LM: GV': -4.0}}


...

    for {0: {'sample': '2JJ-2HB-5MGJJ-MG2'}, 1: {'CM: GV': -2.5, 'RM+LM: GV': -5.5}}
    for {0: {'sample': '2JJ-2HB-5MGJJ-MG2'}, 1: {'CM: GV': -2.5, 'RM+LM: GV': -5.5}}


(12,) [53 83] [16 45 81 93]
(13,) [83] [16 51 82 93]
(14,) [ 5 53 83] [16 52]
(15,) [ 5 50 83] [16]
(16,) [84] [15 45 93]
(17,) [49 70 86] [13 32 45 70 94]
(18,) [87] [12 30 47]
(19,) [89] [10 37 96]
(20,) [ 6 78 91] [7]
(21,) [92] [ 6 82 93]
(22,) [ 4 93] [ 6 17 31]
(23,) [ 4 30 91] [7]
(24,) [89] [10 45]
(25,) [16 50 68 86] [12 71]
(26,) [25 50 64 84] [14 47]
(27,) [49 68 82] [16]
(28,) [51 80] [18 47]
(29,) [50 79] [20 46]
(30,) [ 7 50 78] [21 47]
(31,) [50 78] [21 46 93]
(32,) [41 51 78] [21 48 85 94]
(33,) [ 1 23 50 78] [21 46]
(34,) [49 79] [19 47 87 95]
(35,) [50 81] [18 50 96]
(36,) [50 83] [15 48 94]
(37,) [85] [13 94]
(38,) [ 4 87] [11 50]
(39,) [89] [9]
(40,) [90] [ 9 94]
(41,) [ 5 89] [11]
(42,) [ 4 87] [14 35 50]
(43,) [ 7 49 64 84] [18 33 50]
(44,) [48 62 81] [21 50]
(45,) [48 62 78] [24 50 76]
(46,) [ 9 47 75] [27 51 78 89]
(47,) [ 9 47 73] [29 51]
(48,) [ 8 47 72] [31 52 92]
(49,) [ 8 46 70] [32 51 89]
(50,) [ 8 45 70] [32 51 90]
(51,) [ 7 45 70] [32 52 91]
(52,) [ 7 45

    for {0: {'sample': '2JJ-2HB-5MGJJ-MG2'}, 1: {'CM: GV': -2.5, 'RM+LM: GV': -6.0}}
    for {0: {'sample': '2JJ-2HB-5MGJJ-MG2'}, 1: {'CM: GV': -2.5, 'RM+LM: GV': -6.0}}


(12,) [56 83] [16 44 94]
(13,) [ 6 56 83] [16 42]
(14,) [ 5 55 83] [15 43]
(15,) [ 4 54 84] [14]
(16,) [54 85] [13 43 93]
(17,) [ 4 87] [11]
(18,) [88] [9]
(19,) [ 5 79 91] [8]
(20,) [ 6 22 92] [6]
(21,) [92] [ 6 95]
(22,) [ 5 63 90] [7]
(23,) [ 5 88] [9]
(24,) [70 86] [12]
(25,) [ 5 54 84] [15 30 45 68]
(26,) [ 1 54 63 82] [18 44]
(27,) [54 80] [20 44]
(28,) [55 79] [22 45 70]
(29,) [54 78] [22 43]
(30,) [55 78] [23 44 82 95]
(31,) [ 3 54 78] [23 45]
(32,) [53 78] [22 45]
(33,) [54 79] [20 46 96]
(34,) [53 81] [18 45]
(35,) [54 82] [16 45 96]
(36,) [ 3 52 85] [14]
(37,) [87] [12 39]
(38,) [52 74 88] [10 97]
(39,) [ 5 87] [9]
(40,) [50 85] [11 59 80]
(41,) [51 65 83] [13 47]
(42,) [16 51 80] [17 48]
(43,) [13 51 77] [22 48]
(44,) [13 25 50 75] [26 48 73]
(45,) [ 7 13 49 73] [30 48 74 84]
(46,) [12 49 71] [34 49 85]
(47,) [13 49 69] [36 49 85 96]
(48,) [11 49 69] [39 86 97]
(49,) [12 49 68] [39 86]
(50,) [12 49 68] [40 86]
(51,) [12 48 69] [39 86 93]
(52,) [ 4 11 49 70] [37 50 87]
(53,)

IndexError: index 0 is out of bounds for axis 0 with size 0